# Calibrating using 22-Na and 60-Co

Baseline-subtracted acquisitions were used to determine if 22-Na and 60-Co were siutable peaks for energy calibration. We also included a 137-Cs source in the acquisition, with the intention of using the peak to check the accuracy of the calibration. 

The baseline-subtracted histograms of an acquisition with 2 active channels were obtained. The baseline value for each channel is the mean of the readings from all the events triggered ONLY by external HOLD assertion. This value is subtracted from the values of each "real" event (triggered from internal HOLD).

The dataset for this analysis is a background acquisition of 500 000 events. SIPHRA channels 2 and 14 were connected to SiPM channels 1-4 and 5-8, respectively, using charge comparator, with a threshold value of 20. The source acquisition registered 1 000 000 events. 

In [1]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys
import ipynbname
from pathlib import Path

project_root = str(ipynbname.path().parent.parent)
sys.path.append(project_root)

from processing import SiphraAcquisition, fit_peak_expbg

In [2]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614
CS137_MEV = 0.662
NA22_MEV = 0.511
CO60_MEV_1 = 1.173
CO60_MEV_2 = 1.332
colors = [ROOT.kRed, ROOT.kBlue, ROOT.kGreen, ROOT.kOrange, ROOT.kViolet, ROOT.kYellow, ROOT.kSpring, ROOT.kCyan,]

In [3]:
# Datasets
acq_background = SiphraAcquisition(project_root+'/data/260223/5_SiPM_Ch1-4_Ch2_Ch5-8_Ch14_Active_1-16_PT100_background.csv',
                                   active_chs=[2,14],
                                   exposure_sec=277.478)
acq_sources = SiphraAcquisition(project_root+'/data/260223/3_SiPM_Ch1-4_Ch2_Ch5-8_Ch14_Active_1-16_PT100_Sources_Cs137_Co60_Na22.csv',
                                      active_chs=[2,14],
                                      exposure_sec=448.742)

In [ ]:
if ROOT.gROOT.FindObject('cv'):
    ROOT.gROOT.FindObject('cv').Close()

SUMMED_XR=5000 # Right limit of the summed channel's x-axis

c = ROOT.TCanvas('cv', 'cv', 1200, 800)
ROOT.gStyle.SetOptStat(11)
ROOT.gStyle.SetStatFontSize(0.03)
ROOT.gStyle.SetStatW(0.16)

names=['Background radiation', '22Na and 60Co', 'Background subtracted']
hists = []

lg = ROOT.TLegend(0.48, 0.61, 0.75, 0.83)

Yinit = 0.82 # For stat boxes

for idx, (acq, name) in enumerate(zip([acq_background, acq_sources], names)):
    # print(f"Current file: {acq.filepath.name}")
    ch = 'Summed' #acq.active_chs[0]
    hist = ROOT.TH1F(f"{ch} {name}", "", int(BITS9*SUMMED_XR/BITS12), 0, SUMMED_XR)
    hist.Fill(acq[ch]/len(acq.active_chs))
    hist.Scale(1/acq.exposure)
    hist.GetXaxis().SetTitle("ADC channel number")
    hist.GetYaxis().SetTitle(r"Counts/s")
    hist.SetLineColor(colors[idx])
    hist.SetTitle("Background source subtraction")
    # hist.GetYaxis().SetRangeUser(0, 1e4)
    lg.SetHeader("SIPHRA Channel")
    lg.AddEntry(hist, f"{ch} {name}", 'l')
    hists.append(hist)
    hist.Draw('sames hist')
    
idx+=1
h_diff = hists[1].Clone("Background subtracted")
h_diff.Add(hists[0], -1)
h_diff.SetLineColor(colors[idx])
lg.AddEntry(h_diff, f"{ch} {name}", 'l')
hists.append(h_diff)
h_diff.Draw('sames hist')
c.SetLogy()
ROOT.gPad.Update()
for i, sp in enumerate(hists):
    st = sp.FindObject("stats")
    # print(type(st))
    st.SetY1NDC(Yinit - i*0.08)
    st.SetY2NDC(0.1)
lg.Draw()
c.Draw()
ROOT.gPad.Modified()
ROOT.gPad.Update()



# Bad resolution

It seems like the energy peaks from $^{60}$Co, at energies 1.173 MeV and 1.332 MeV are too close to be distinguishable, making them bad candidates for calibration. The resolution is pretty bad, which is probably not good. 

$^{22}$Na has a second peak in between these peaks, at 1.275 MeV. Is this peak showing up stronger than expected, causing issues?
How can we get better resolution?
